In [1]:
import pandas as pd

# parquet 파일 경로 지정
file_path = 'singleop.parquet'

# pyarrow(또는 fastparquet) 기반으로 parquet 파일 읽기
df = pd.read_parquet(file_path)

In [2]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,input,output_program,output_answer,split,dataset
0,consensus,Solve the following math word problem.\n\n Mis...,Solve the following math word problem.\n\n Mis...,[13],[13],math,Misha has 34 dollars. How many more dollars d...,answer=(47.0-34.0)\nprint(answer),13,test,singleop
1,consensus,Solve the following math word problem.\n\nIt t...,Solve the following math word problem.\n\nIt t...,[10],[10],math,It took Amanda 5 hours to stroll to Kimberly's...,answer=(5.0*2.0)\nprint(answer),10,test,singleop
2,consensus,Solve the following math word problem.\n\nAnge...,Solve the following math word problem.\n\nAnge...,[15],[15],math,Angela has 11 tickets. Annie gives Angela 4 mo...,answer=(11.0+4.0)\nprint(answer),15,test,singleop
3,consensus,Solve the following math word problem.\n\nHenr...,Solve the following math word problem.\n\nHenr...,[7],[7],math,Henry wants to split a collection of crayons i...,answer=(42.0/6.0)\nprint(answer),7,test,singleop
4,consensus,Solve the following math word problem.\n\n Ies...,Solve the following math word problem.\n\n Ies...,[208],[208],math,Iesha has 344 books. 136 are about school and...,answer=(344.0-136.0)\nprint(answer),208,test,singleop


In [3]:
df['platinum_prompt'][0]

'Solve the following math word problem.\n\n Misha has 34 dollars. How many more dollars does she have to earn to have 47 dollars?\n\nThink step-by-step. Then, provide the final answer as a single number in the format "Answer: XXX" with no extra formatting.'

In [4]:
import pandas as pd
import re

# 1. Read the question again: ...
def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2'] = df['platinum_prompt'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Think step-by-step)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

# 2. Summarize the question first.
df['sum'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Summarize the question first. ",
    regex=True
)

# 3. Organize the question into a table first.
df['table'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Organize the question into a table first. ",
    regex=True
)

# 4. Organize the question into a graph structure first.
df['graph'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Organize the question into a graph structure first. ",
    regex=True
)

# 5. Summarize the question in bullet points first.
df['bullet_point'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Summarize the question in bullet points first. ",
    regex=True
)

# 6. Read the question again first.
df['sRE2'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Read the question again first. ",
    regex=True
)

In [5]:
df['RE2'][0]

'Solve the following math word problem.\n\n Misha has 34 dollars. How many more dollars does she have to earn to have 47 dollars?\n\nRead the question again:  Misha has 34 dollars. How many more dollars does she have to earn to have 47 dollars?\n\nThink step-by-step. Then, provide the final answer as a single number in the format "Answer: XXX" with no extra formatting.'

In [6]:
df['platinum_prompt_no_cot'][0]

'Solve the following math word problem.\n\n Misha has 34 dollars. How many more dollars does she have to earn to have 47 dollars?\n\nThen, provide the final answer as a single number in the format "Answer: XXX" with no extra formatting.'

In [7]:
import pandas as pd
import re

def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Then, provide the final answer as a single number)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

df['sum_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Summarize the question first. ",
    regex=True
)

df['table_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Organize the question into a table first. ",
    regex=True
)

df['graph_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Organize the question into a graph structure first. ",
    regex=True
)

df['bullet_point_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Summarize the question in bullet points first. ",
    regex=True
)

df['sRE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Read the question again first. ",
    regex=True
)

In [13]:
df['graph_no_cot'][0]

'Solve the following math word problem.\n\n Misha has 34 dollars. How many more dollars does she have to earn to have 47 dollars?\n\nOrganize the question into a graph structure first. Then, provide the final answer as a single number in the format "Answer: XXX" with no extra formatting.'

In [14]:
df.to_parquet('graph_add_task.parquet')

In [15]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,input,output_program,output_answer,split,...,table,graph,bullet_point,sRE2,RE2_no_cot,sum_no_cot,table_no_cot,graph_no_cot,bullet_point_no_cot,sRE2_no_cot
0,consensus,Solve the following math word problem.\n\n Mis...,Solve the following math word problem.\n\n Mis...,[13],[13],math,Misha has 34 dollars. How many more dollars d...,answer=(47.0-34.0)\nprint(answer),13,test,...,Solve the following math word problem.\n\n Mis...,Solve the following math word problem.\n\n Mis...,Solve the following math word problem.\n\n Mis...,Solve the following math word problem.\n\n Mis...,Solve the following math word problem.\n\n Mis...,Solve the following math word problem.\n\n Mis...,Solve the following math word problem.\n\n Mis...,Solve the following math word problem.\n\n Mis...,Solve the following math word problem.\n\n Mis...,Solve the following math word problem.\n\n Mis...
1,consensus,Solve the following math word problem.\n\nIt t...,Solve the following math word problem.\n\nIt t...,[10],[10],math,It took Amanda 5 hours to stroll to Kimberly's...,answer=(5.0*2.0)\nprint(answer),10,test,...,Solve the following math word problem.\n\nIt t...,Solve the following math word problem.\n\nIt t...,Solve the following math word problem.\n\nIt t...,Solve the following math word problem.\n\nIt t...,Solve the following math word problem.\n\nIt t...,Solve the following math word problem.\n\nIt t...,Solve the following math word problem.\n\nIt t...,Solve the following math word problem.\n\nIt t...,Solve the following math word problem.\n\nIt t...,Solve the following math word problem.\n\nIt t...
2,consensus,Solve the following math word problem.\n\nAnge...,Solve the following math word problem.\n\nAnge...,[15],[15],math,Angela has 11 tickets. Annie gives Angela 4 mo...,answer=(11.0+4.0)\nprint(answer),15,test,...,Solve the following math word problem.\n\nAnge...,Solve the following math word problem.\n\nAnge...,Solve the following math word problem.\n\nAnge...,Solve the following math word problem.\n\nAnge...,Solve the following math word problem.\n\nAnge...,Solve the following math word problem.\n\nAnge...,Solve the following math word problem.\n\nAnge...,Solve the following math word problem.\n\nAnge...,Solve the following math word problem.\n\nAnge...,Solve the following math word problem.\n\nAnge...
3,consensus,Solve the following math word problem.\n\nHenr...,Solve the following math word problem.\n\nHenr...,[7],[7],math,Henry wants to split a collection of crayons i...,answer=(42.0/6.0)\nprint(answer),7,test,...,Solve the following math word problem.\n\nHenr...,Solve the following math word problem.\n\nHenr...,Solve the following math word problem.\n\nHenr...,Solve the following math word problem.\n\nHenr...,Solve the following math word problem.\n\nHenr...,Solve the following math word problem.\n\nHenr...,Solve the following math word problem.\n\nHenr...,Solve the following math word problem.\n\nHenr...,Solve the following math word problem.\n\nHenr...,Solve the following math word problem.\n\nHenr...
4,consensus,Solve the following math word problem.\n\n Ies...,Solve the following math word problem.\n\n Ies...,[208],[208],math,Iesha has 344 books. 136 are about school and...,answer=(344.0-136.0)\nprint(answer),208,test,...,Solve the following math word problem.\n\n Ies...,Solve the following math word problem.\n\n Ies...,Solve the following math word problem.\n\n Ies...,Solve the following math word problem.\n\n Ies...,Solve the following math word problem.\n\n Ies...,Solve the following math word problem.\n\n Ies...,Solve the following math word problem.\n\n Ies...,Solve the following math word problem.\n\n Ies...,Solve the following math word problem.\n\n Ies...,Solve the following math word problem.\n\n Ies...
